# Laboratorio 7: Sistemas de Recomendación y filtrado colaborativo

In [1]:
import numpy as np
import pandas as pd
import re
import functions as assets


In [2]:
import warnings
warnings.filterwarnings('ignore')

We'll build our X and Y matrixes to pass them though a Adam model

In [3]:
links_small = pd.read_csv('links_small.csv')
ratings = pd.read_csv('ratings_small.csv')
user_ratings = links_small.merge(ratings, how='left', on='movieId')
user_ratings.drop(['timestamp'], inplace=True, axis=1)
user_ratings['rating'].fillna(0, inplace=True)
user_ratings['rating'] = user_ratings['rating'].astype(int)
user_ratings.dropna(inplace=True, subset=['userId'])

## User rating (X) matrix

In [4]:
user_ratings.head(3)


,movieId,imdbId,tmdbId,userId,rating
0,1,114709,862.0,7.0,3
1,1,114709,862.0,9.0,4
2,1,114709,862.0,13.0,5


In [5]:
equis = user_ratings.pivot_table(index='tmdbId', columns='userId', values='rating', fill_value=0)
equis.head(3)

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,662.0,663.0,664.0,665.0,666.0,667.0,668.0,669.0,670.0,671.0
tmdbId,,,,,,,,,,,,,,,,,,,,,
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x = equis.to_numpy()
print(x.shape)

(9053, 671)


## Movie feature (Y) matrix

In [7]:
movies_meta = pd.read_csv('movies_metadata.csv')
movies_md = movies_meta[['genres', 'id', 'imdb_id']]
movies_md.shape

(45466, 3)

In [8]:
movies_md.head(3)

,genres,id,imdb_id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228


In [9]:
works_data = pd.DataFrame(movies_md.genres.str.split(',').tolist(), index=movies_md.id).stack()
works_data = works_data.reset_index([0, 'id'])
works_data.columns = ['id', 'value']
works_data['id_genre'] = works_data.value.str.extract("{'id': (\d+)", expand=False)
works_data.drop(['value'], axis=1, inplace=True)
works_data.dropna(inplace=True)

In [10]:
works_data.head(3)

,id,id_genre
0,862,16
2,862,35
4,862,10751


In [11]:
works_data['ones'] = np.ones(works_data.shape[0])
ye = works_data.pivot_table(index='id', columns='id_genre', values='ones', fill_value=0)
y = ye[:9053]
ye.head(3)

id_genre,10402,10749,10751,10752,10769,10770,12,14,16,18,27,28,35,36,37,53,80,878,9648,99
id,,,,,,,,,,,,,,,,,,,,
100,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
10000,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
10001,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [12]:
print(y.shape)

(9053, 20)


## Apply Adam model

We're going to use only X matrix because we're going to use collaborative filtering (calculates its own Y matrix)

In [13]:
# filas son peliculas y columnas son usuarios
n, m = x.shape
f = y.shape[1]
Y = np.random.rand(n*f).reshape(n, f)
theta_0 = np.random.rand(f, m)

In [14]:
# 70% train, 15% test, 15% cv
X_train = x[:6337]
X_cv = x[6338:7695]
X_test = x[7696:9052]
y_train = Y[:6337]
y_cv = Y[6338:7695]
y_test = Y[7696:9052]

Model 1

In [15]:
X_f, T_f, values, jac_norms = assets.adam(y_train, X_train, theta_0, assets.cost, assets.gradient)

1.0 	 loss=42,111.45


UFuncTypeError: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

Model 2

In [ ]:
X_f, T_f, values, jac_norms = assets.adam(y_train, X_train ** 2, theta_0, assets.cost, assets.gradient)